## Data

In [8]:
import pandas as pd
import numpy as np

def prepare_multistate_data(df, covariate_cols=None):
    """
    Prepare multistate data with counterfactuals, preserving covariates.
    
    Parameters:
    -----------
    df : DataFrame
        Input data with columns: patnr, time, status, and any covariates
    covariate_cols : list, optional
        List of covariate column names to preserve. If None, will auto-detect.
    """
    transitions = df.copy()
    
    # Auto-detect covariates if not specified
    if covariate_cols is None:
        # Assume columns that are not patnr, time, or status are covariates
        standard_cols = ['patnr', 'time', 'status']
        covariate_cols = [col for col in df.columns if col not in standard_cols]
    
    # Add tstart column
    transitions['tstart'] = 0
    # Add origin state column
    transitions['origin_state'] = 0
    # Rename columns
    transitions.rename(columns={'time': 'tstop', 'status':'target_state'}, inplace=True)
    # Add censoring column
    transitions["status"] = np.where(
        transitions["target_state"] == 0, 0, 1
    )
    # Add dummy target state to censored observations
    transitions.loc[transitions["target_state"] == 0, "target_state"] = 1 

    # Add counterfactuals for each observation
    counterfactuals = []
    for _, row in transitions.iterrows():
        # For each transition to state k, add counterfactuals to all other possible states
        possible_states = [1, 2]  # Adjust based on your actual states
        current_target = row['target_state']
        
        for state in possible_states:
            if state != current_target:
                # Create counterfactual transition
                counterfactual = row.copy()
                counterfactual['target_state'] = state
                counterfactual['status'] = 0  # Counterfactuals are censored
                counterfactuals.append(counterfactual)

    # Combine original transitions with counterfactuals
    if counterfactuals:
        counterfactuals_df = pd.DataFrame(counterfactuals)
        transitions = pd.concat([transitions, counterfactuals_df], ignore_index=True)

    # Sort by patient number and target state for clarity
    transitions = transitions.sort_values(['patnr', 'target_state']).reset_index(drop=True)
    
    # Return all columns including covariates
    base_cols = ["patnr", "tstart", "tstop", "origin_state", "target_state", "status"]
    return transitions[base_cols + covariate_cols]


def counterfactual_to_standard_format(df, covariate_cols=None):
    """
    Convert counterfactual data to standard competing risks format with covariates.
    
    Parameters:
    -----------
    df : DataFrame
        Counterfactual format data
    covariate_cols : list, optional
        List of covariate column names to preserve
    """
    # Auto-detect covariates if not specified
    if covariate_cols is None:
        standard_cols = ['patnr', 'tstart', 'tstop', 'origin_state', 'target_state', 'status']
        covariate_cols = [col for col in df.columns if col not in standard_cols]
    
    # Get only the rows where an event actually occurred
    actual_events = df[df['status'] == 1].copy()
    
    # If a patient has no events (all status=0), they're censored
    all_patients = df['patnr'].unique()
    patients_with_events = actual_events['patnr'].unique()
    censored_patients = set(all_patients) - set(patients_with_events)
    
    # Add censored patients
    if censored_patients:
        censored_data = []
        for patient in censored_patients:
            patient_data = df[df['patnr'] == patient].iloc[0].copy()
            patient_data['status'] = 0
            patient_data['target_state'] = 0  # 0 typically indicates censoring
            censored_data.append(patient_data)
        
        censored_df = pd.DataFrame(censored_data)
        actual_events = pd.concat([actual_events, censored_df], ignore_index=True)
    
    # Rename columns to match expected format
    standard_df = actual_events.rename(columns={
        'patnr': 'id',
        'tstop': 'duration', 
        'target_state': 'event' 
    })
    
    # Keep necessary columns including covariates
    base_cols = ['id', 'duration', 'event']
    standard_df = standard_df[base_cols + covariate_cols]
    
    return standard_df


# Example usage with aidssi dataset
# First, let's check what columns are in aidssi
aidssi = pd.read_csv("/home/azureuser/cloudfiles/code/Users/draetta.edoardo/pymsm/src/pymsm/datasets/aidssi.csv", index_col=0)
print("Original columns:", aidssi.columns.tolist())
display(aidssi.head())

# Identify covariate columns (adjust based on your actual dataset)
covariate_cols = [col for col in aidssi.columns if col not in ['patnr', 'time', 'status', 'cause']]
print("Detected covariates:", covariate_cols)

# Prepare data with covariates
transitions = prepare_multistate_data(aidssi, covariate_cols=covariate_cols)
print("\nTransitions data shape:", transitions.shape)
print("Transitions columns:", transitions.columns.tolist())
display(transitions.head())

# Convert to standard format preserving covariates
standard_df = counterfactual_to_standard_format(transitions, covariate_cols=covariate_cols)
print("\nStandard format shape:", standard_df.shape)
print("Standard format columns:", standard_df.columns.tolist())
display(standard_df.head())

Original columns: ['patnr', 'time', 'status', 'cause', 'ccr5']


,patnr,time,status,cause,ccr5
1,1,9.106,1,AIDS,WW
2,2,11.039,0,event-free,WM
3,3,2.234,1,AIDS,WW
4,4,9.878,2,SI,WM
5,5,3.819,1,AIDS,WW


Detected covariates: ['ccr5']

Transitions data shape: (658, 7)
Transitions columns: ['patnr', 'tstart', 'tstop', 'origin_state', 'target_state', 'status', 'ccr5']


,patnr,tstart,tstop,origin_state,target_state,status,ccr5
0,1,0,9.106,0,1,1,WW
1,1,0,9.106,0,2,0,WW
2,2,0,11.039,0,1,0,WM
3,2,0,11.039,0,2,0,WM
4,3,0,2.234,0,1,1,WW



Standard format shape: (329, 4)
Standard format columns: ['id', 'duration', 'event', 'ccr5']


,id,duration,event,ccr5
0,1,9.106,1,WW
1,3,2.234,1,WW
2,4,9.878,2,WM
3,5,3.819,1,WW
4,6,6.801,1,WW


In [2]:
from pymsm.visualization import StateDiagramGenerator

state_labels = {0: "Event-free", 1: "AIDS", 2: "SI"}
terminal_states = [3]

sdg = StateDiagramGenerator(
    dataset=transitions,
    patient_id='patnr',
    from_state='origin_state',
    to_state='target_state',
    tstart='tstart',
    tstop='tstop',
    status='status',
    state_labels=state_labels,
    terminal_states=terminal_states
)

sdg.plot_state_diagram()

## CompetingRisksModels

In [3]:
from pymsm.models import CompetingRisksModel

crm = CompetingRisksModel()
crm.base_estimator

CoxPHSurvivalAnalysis()

In [4]:
model_df = standard_df.copy()
model_df["ccr5"] = (model_df["ccr5"] == "WW").astype(int)
model_df.head()

,id,duration,event,ccr5
0,1,9.106,1,1
1,3,2.234,1,1
2,4,9.878,2,0
3,5,3.819,1,1
4,6,6.801,1,1


In [5]:
from pymsm.models import CompetingRisksModel

# Now you can use it with CompetingRisksModel
crm = CompetingRisksModel()
crm.fit(model_df, duration_col='duration', event_col='event')

CompetingRisksModel(base_estimator=CoxPHSurvivalAnalysis())

In [6]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from pymsm.models import CompetingRisksModel
from sksurv.metrics import concordance_index_censored, cumulative_dynamic_auc, brier_score
import matplotlib.pyplot as plt


# Create train-test split
train_df, test_df = train_test_split(
    model_df, test_size=0.2, random_state=42
)

# Fit the model
crm = CompetingRisksModel()
crm.fit(train_df, duration_col='duration', event_col='event')

CompetingRisksModel(base_estimator=CoxPHSurvivalAnalysis())

In [ ]:
# Define evaluation time points
times = np.percentile(train_df['duration'], np.linspace(10, 90, 10))


# Create a comprehensive scoring function
def score_competing_risks_model(model, test_df, train_df, times, duration_col='duration', event_col='event'):
    """
    Comprehensive scoring for competing risks models.
    
    Returns:
    --------
    scores : dict
        Dictionary containing various performance metrics
    """
    scores = {}
    
    # Get features
    X_test = test_df.drop([duration_col, event_col], axis=1)
    X_train = train_df.drop([duration_col, event_col], axis=1)
    
    # Get event types
    event_types = model.event_types_
    
    # For each event type, calculate metrics
    for event_type in event_types:
        print(f"Scoring for Event Type {event_type}:")
        scores[f'event_{event_type}'] = {}
        
        # 1. Concordance Index (C-index)
        # Get risk scores (using cumulative hazard at median time)
        median_time = np.median(train_df[duration_col])
        risk_scores = model.predict_cif(X_test, [median_time], event_type)[:, 0]
        
        # Create binary event indicator for this specific event
        event_indicator = (test_df[event_col] == event_type).astype(int)
        
        # Calculate C-index
        c_index, _, _, _, _ = concordance_index_censored(
            event_indicator.astype(bool),
            test_df[duration_col],
            risk_scores
        )
        scores[f'event_{event_type}']['c_index'] = c_index
        print(f"  C-index: {c_index:.3f}")
    return scores

# Score the model
scores = score_competing_risks_model(crm, test_df, train_df, times)

Scoring for Event Type 2:
  C-index: 0.486
Scoring for Event Type 1:
  C-index: 0.624


In [ ]:

# Cross-validation function for more robust evaluation
from sklearn.model_selection import StratifiedKFold

def cross_validate_competing_risks(df, n_splits=5, random_state=42):
    """
    Perform cross-validation for competing risks model.
    """
    # Prepare data
    X = df.drop(['duration', 'event'], axis=1)
    y_duration = df['duration']
    y_event = df['event']
    
    # Create stratified folds based on event type
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    
    cv_scores = []
    
    for fold, (train_idx, test_idx) in enumerate(skf.split(X, y_event)):
        print(f"\nFold {fold + 1}/{n_splits}")
        
        # Split data
        train_df = df.iloc[train_idx]
        test_df = df.iloc[test_idx]
        
        # Fit model
        crm = CompetingRisksModel()
        crm.fit(train_df, duration_col='duration', event_col='event')
        
        # Define evaluation times based on training data
        times = np.percentile(
            train_df['duration'], 
            np.linspace(10, 90, 10)
        )
        
        # Score model
        fold_scores = score_competing_risks_model(
            crm, test_df, train_df, times
        )
        cv_scores.append(fold_scores)
    
    # Aggregate results
    aggregated_scores = {}
    
    for event_key in cv_scores[0].keys():
        aggregated_scores[event_key] = {
            'c_index_mean': np.mean([s[event_key]['c_index'] for s in cv_scores]),
            'c_index_std': np.std([s[event_key]['c_index'] for s in cv_scores]),
        }
    
    return aggregated_scores

# Perform cross-validation
cv_results = cross_validate_competing_risks(model_df)

# Print cross-validation results
print("\n=== Cross-Validation Results ===")
for event_key, metrics in cv_results.items():
    print(f"\n{event_key}:")
    print(f"  C-index: {metrics['c_index_mean']:.3f} ± {metrics['c_index_std']:.3f}")
    # print(f"  Mean TD-AUC: {metrics['mean_td_auc_mean']:.3f} ± {metrics['mean_td_auc_std']:.3f}")
    # print(f"  IBS: {metrics['ibs_mean']:.3f} ± {metrics['ibs_std']:.3f}")



Fold 1/5
Scoring for Event Type 1:
  C-index: 0.586
Scoring for Event Type 2:
  C-index: 0.602

Fold 2/5
Scoring for Event Type 1:
  C-index: 0.520
Scoring for Event Type 2:
  C-index: 0.635

Fold 3/5
Scoring for Event Type 1:
  C-index: 0.534
Scoring for Event Type 2:
  C-index: 0.582

Fold 4/5
Scoring for Event Type 1:
  C-index: 0.598
Scoring for Event Type 2:
  C-index: 0.463

Fold 5/5
Scoring for Event Type 1:
  C-index: 0.555
Scoring for Event Type 2:
  C-index: 0.540

=== Cross-Validation Results ===

event_1:
  C-index: 0.558 ± 0.030

event_2:
  C-index: 0.564 ± 0.059


In [26]:
from sksurv.ensemble import RandomSurvivalForest

crm = CompetingRisksModel(base_estimator=RandomSurvivalForest())
crm.fit(train_df, duration_col='duration', event_col='event')

CompetingRisksModel(base_estimator=RandomSurvivalForest())

In [27]:
crm.event_models_

{2: RandomSurvivalForest(), 1: RandomSurvivalForest()}

In [34]:
crm.event_models_[1].estimators_[0].tree_

In [42]:
surv_tree = crm.event_models_[1].estimators_[0]
tree_struct = surv_tree.tree_

print("Number of nodes:", tree_struct.node_count)
print("Children left:", tree_struct.children_left)
print("Children right:", tree_struct.children_right)
print("Feature used at each node:", tree_struct.feature)
print("Threshold at each node:", tree_struct.threshold)


Number of nodes: 73
Children left: [ 1  2  3 -1  5  6 -1 -1  9 -1 -1 -1 13 -1 15 16 17 18 19 -1 21 22 -1 -1
 25 -1 27 28 29 30 31 -1 -1 -1 -1 36 -1 -1 39 -1 -1 42 -1 -1 45 -1 47 -1
 -1 -1 51 -1 53 -1 55 56 -1 -1 59 -1 61 -1 63 64 -1 -1 67 68 69 -1 -1 -1
 -1]
Children right: [12 11  4 -1  8  7 -1 -1 10 -1 -1 -1 14 -1 50 49 44 41 20 -1 24 23 -1 -1
 26 -1 38 35 34 33 32 -1 -1 -1 -1 37 -1 -1 40 -1 -1 43 -1 -1 46 -1 48 -1
 -1 -1 52 -1 54 -1 58 57 -1 -1 60 -1 62 -1 66 65 -1 -1 72 71 70 -1 -1 -1
 -1]
Feature used at each node: [ 1  0  0 -2  0  0 -2 -2  0 -2 -2 -2  0 -2  0  0  0  0  0 -2  0  0 -2 -2
  0 -2  0  0  0  0  0 -2 -2 -2 -2  0 -2 -2  0 -2 -2  0 -2 -2  0 -2  0 -2
 -2 -2  0 -2  0 -2  0  0 -2 -2  0 -2  0 -2  0  0 -2 -2  0  0  0 -2 -2 -2
 -2]
Threshold at each node: [  0.5 240.5  68.5  -2.  115.  100.5  -2.   -2.  184.5  -2.   -2.   -2.
  22.5  -2.  214.  204.  181.  153.5  41.   -2.   65.5  49.   -2.   -2.
  77.   -2.  134.5 117.5 107.  103.   85.5  -2.   -2.   -2.   -2.  125.5
  -2.   -